In [1]:
import pandas as pd
import sys
sys.path.append('../')
from potnet import *

This example demonstrates the workflow for using $\texttt{POTNet}$ for tabular data generation of heterogeneous feature types. 

In [39]:
data = pd.read_csv('../data/credit_card.csv', header=0)
cat_cols = ['SEX', 'EDUCATION', 'MARRIAGE', 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6']
print(data.shape)
data.head()


(1000, 24)


,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,50000,2,2,2,29,2,2,2,2,2,...,28768,29639,28629,0,3304,4,2230,0,2009,1
1,70000,2,2,1,46,0,0,0,0,0,...,59665,29510,26738,2600,3700,2600,1200,1200,1000,0
2,160000,2,1,2,27,0,0,-2,-1,-1,...,15866,4168,2067,0,6000,15866,4168,2067,11826,0
3,310000,1,2,2,30,0,0,0,0,0,...,245884,200380,221201,10400,11200,10840,9361,5300,8205,0
4,230000,1,1,1,45,0,0,0,0,0,...,29202,29683,30277,2200,2000,1100,1000,980,1012,0


In [3]:
potnet_model= POTNet(embedding_dim=data.shape[1],
                      categorical_cols=cat_cols,
                      numeric_output_data_type = 'integer', # discrete data
                      epochs=1000,
                      batch_size=64 # small batchsize because number of samples is small
                      )
potnet_model.fit(data)
potnet_model.save('potnet_model.pt')

Training POTNet:   0%|          | 0/1000 [00:00<?, ?it/s]

Model saved to potnet_model.pt


In [38]:
num_samps = data.shape[0]
gen_data = potnet_model.generate(num_samps)
# Dataset specific post-processing
# Since the `LIMIT_BAL` feature is rounded to the nearest 10000 in the original data, we will do the same for the generated data
gen_data['LIMIT_BAL'] = gen_data['LIMIT_BAL'].round(-4)
gen_data.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,220000.0,2,1,1,34.0,-2,-2,-2,-2,-2,...,7379.0,6801.0,4185.0,3239.0,1019.0,1447.0,643.0,2789.0,111.0,0.0
1,30000.0,2,2,2,25.0,0,0,0,0,0,...,21993.0,22823.0,19900.0,3050.0,2245.0,2232.0,2097.0,3907.0,1544.0,0.0
2,60000.0,2,2,2,25.0,1,2,0,0,0,...,26245.0,27831.0,25844.0,2445.0,1368.0,1722.0,2927.0,2736.0,2380.0,0.0
3,250000.0,2,2,1,33.0,0,0,0,0,0,...,156248.0,156019.0,148570.0,8408.0,9081.0,7551.0,6622.0,7397.0,6368.0,0.0
4,80000.0,2,3,1,44.0,0,0,0,0,0,...,19421.0,22650.0,23606.0,2178.0,2393.0,2250.0,787.0,2839.0,621.0,0.0
